## Aggregate Function Example
> Time-series aggregation function   
  performs a rolling aggregation on {df_artifact}, over {window} by the selected {keys}
  applying {metric_aggs} on {metrics} and {label_aggs} on {labels}. adding {suffix} to the 
  feature names.
    
    

In [1]:
# Make sure mlrun is aligned !
# !/User/align_mlrun.sh
# !pip install sklearn

In [2]:
import pandas as pd
import mlrun
from sklearn.datasets import load_iris
mlrun.set_environment(project='function-marketplace')

> 2021-10-05 10:40:16,310 [info] loaded project function-marketplace from MLRun DB


('function-marketplace', 'v3io:///projects/{{run.project}}/artifacts')

### Simple aggregations on sklearn's iris dataset

In [3]:
dataset = load_iris()
df = pd.DataFrame(dataset['data'],columns = dataset['feature_names'])
df['target'] = [dataset['target_names'][i] for i in dataset['target']]
df.to_parquet("iris.parquet")
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
# import the function
aggregate_function = mlrun.import_function("hub://aggregate").apply(mlrun.auto_mount())

In [5]:
import os
os.path.abspath('iris.parquet')

'/User/test/functions/aggregate/iris.parquet'

In [6]:
aggregate_run = aggregate_function.run(params={'metrics': dataset['feature_names'][0],
               'metric_aggs': ['mean', 'sum'],
               'suffix': 'aggregated',
               'inplace': False,
               'window': 2}           
           , inputs={'df_artifact': os.path.abspath('iris.parquet')}
           )

> 2021-10-05 10:40:16,613 [info] starting run aggregate-aggregate uid=c531676eb37a498fb6fe8f867b9d9a7d DB=http://mlrun-api:8080
> 2021-10-05 10:40:16,753 [info] Job is running in the background, pod: aggregate-aggregate-9vqmq
> 2021-10-05 10:40:22,212 [info] Aggregating /User/test/functions/aggregate/iris.parquet
> 2021-10-05 10:40:22,243 [info] Logging artifact
> 2021-10-05 10:40:22,397 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...7b9d9a7d,0,Oct 05 10:40:22,completed,aggregate-aggregate,v3io_user=danikind=jobowner=danihost=aggregate-aggregate-9vqmq,df_artifact,"metrics=sepal length (cm)metric_aggs=['mean', 'sum']suffix=aggregatedinplace=Falsewindow=2",,aggregate


> 2021-10-05 10:40:22,907 [info] run executed, status=completed


### Show results

In [7]:
aggregate_run.artifact('aggregate').as_df().head().style.set_caption("Aggregated Dataframe !")

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,m_e_a_n_aggregated,s_u_m_aggregated
1,4.900000,3.000000,1.400000,0.200000,setosa,5.000000,10.000000
2,4.700000,3.200000,1.300000,0.200000,setosa,4.800000,9.600000
3,4.600000,3.100000,1.500000,0.200000,setosa,4.650000,9.300000
4,5.000000,3.600000,1.400000,0.200000,setosa,4.800000,9.600000
5,5.400000,3.900000,1.700000,0.400000,setosa,5.200000,10.400000


### More complex aggregations

In [8]:
# Getting & inspecting original dataset
DATA = "https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq"
original = pd.read_parquet(DATA)
original.head().style.set_caption("Original Dataframe !")

In [9]:
aggregate_run = aggregate_function.run(params={'metrics': ['cpu_utilization'],
               'labels': ['is_error'],
               'metric_aggs': ['mean', 'sum'],
               'label_aggs': ['max'],
               'suffix': 'daily',
               'inplace': False,
               'window': 5,
               'center': True,               
               'files_to_select': 2}           
           , inputs={'df_artifact': DATA}
           )
    

> 2021-10-05 10:40:24,797 [info] starting run aggregate-aggregate uid=a874ee2e49894f98adc092aa8138db53 DB=http://mlrun-api:8080
> 2021-10-05 10:40:24,931 [info] Job is running in the background, pod: aggregate-aggregate-mpf4p
> 2021-10-05 10:40:30,245 [info] Aggregating https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq
> 2021-10-05 10:40:31,547 [info] Logging artifact
> 2021-10-05 10:40:31,708 [info] run executed, status=completed
Converting input from bool to <class 'numpy.uint8'> for compatibility.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...8138db53,0,Oct 05 10:40:30,completed,aggregate-aggregate,v3io_user=danikind=jobowner=danihost=aggregate-aggregate-mpf4p,df_artifact,"metrics=['cpu_utilization']labels=['is_error']metric_aggs=['mean', 'sum']label_aggs=['max']suffix=dailyinplace=Falsewindow=5center=Truefiles_to_select=2",,aggregate


> 2021-10-05 10:40:34,126 [info] run executed, status=completed


Convert the code to an MLRun function

### Show results

In [10]:
aggregate_run.artifact('aggregate').as_df().head().style.set_caption("Aggregated Dataframe !")